In [1]:
import pandas as pd

In [4]:
csv_file_path = "../data/raw/NYC_Vehicle_Collisions_Crashes.csv"

crash_df = pd.read_csv(csv_file_path)

parquet_file_path = "../data/raw/NYC_Vehicle_Collisions_Crashes.parquet"

crash_df.to_parquet(parquet_file_path, index = False)

C:\Users\61435\AppData\Local\Temp\ipykernel_7624\3212012800.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  crash_df = pd.read_csv(csv_file_path)


ArrowInvalid: ("Could not convert '11226' with type str: tried to convert to double", 'Conversion failed for column ZIP CODE with type object')